## 使用 tf.data.Dataset 加载数据

In [20]:
import tensorflow as tf

In [21]:
tf.__version__

'2.16.1'

### List 列表数据

In [22]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
for element in dataset:
    print(element)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)


2024-04-15 17:56:16.946771: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Generator 生成器

In [23]:
import itertools

def gen():
    for i in itertools.count(1):
        yield (i, [1] * i)

In [24]:
'''
from_generator：创建一个数据集
def from_generator(
      generator,
      output_types=None,
      output_shapes=None,...）

(tf.TensorShape([]), tf.TensorShape([None]))：生成的数据的Tensor形状
tf.TensorShape([]): 表示一个标量（没有维度的数组）
tf.TensorShape([None]): 表示一个可变长度的一维数组
对应生成器函数返回的元组中的两个元素：第一个元素是一个整数，没有维度，第二个元素是一个列表，其长度可以变化
      
'''
dataset = tf.data.Dataset.from_generator(
    gen,
    (tf.int64, tf.int64),
    (tf.TensorShape([]), tf.TensorShape([None])))

print(type(dataset)) # <class 'tensorflow.python.data.ops.flat_map_op._FlatMapDataset'>
print(type(dataset.take(3))) # <class 'tensorflow.python.data.ops.take_op._TakeDataset'>

<class 'tensorflow.python.data.ops.flat_map_op._FlatMapDataset'>
<class 'tensorflow.python.data.ops.take_op._TakeDataset'>


In [25]:
'''
dataset.take(n)方法用于从数据集中取出前n个元素
as_numpy_iterator()方法则允许你迭代这个数据集，并将其中的Tensor对象转换为NumPy数组
list(...)：将迭代器中的所有元素收集到一个Python列表中。
'''

list(dataset.take(3).as_numpy_iterator())

# [(1, array([1])), (2, array([1, 1])), (3, array([1, 1, 1]))]

2024-04-15 17:56:17.340408: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


[(1, array([1])), (2, array([1, 1])), (3, array([1, 1, 1]))]

### 文本文件

In [26]:
parent_dir = "files"
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

In [27]:
import os

'''
tf.cast(index, tf.int64): 将一个Tensor从一种数据类型转换为另一种数据类型。在你的代码示例中，tf.cast(index, tf.int64)的作用是将index变量（通常是Python中的整数类型）转换为TensorFlow的int64类型。
>>> tf.cast(1, tf.int64)
<tf.Tensor: shape=(), dtype=int64, numpy=1>
'''
def labeler(example, index):
    return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

'''
tf.data.TextLineDataset:函数创建一个数据集，它会读取指定文件路径中的文本行。os.path.join(parent_dir, file_name)用于生成每个文件的完整路径。
labeled_dataset：使用map方法将labeler函数应用到lines_dataset数据集中的每个元素（即每一行文本）上。这样，每一行文本都会与相应的标签（文件的索引）关联起来。

'''
for i, file_name in enumerate(FILE_NAMES):
    lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
    labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
    labeled_data_sets.append(labeled_dataset)

In [28]:
'''
这些步骤将所有标记数据集合并成一个单一的数据集，并确保在训练模型时，数据是随机的，每个批次包含64个样本，且数据在每个epoch开始时被打乱一次。
这样的数据预处理对于训练高效的机器学习模型是非常重要的，因为它有助于模型更好地泛化到未见过的数据上。
'''


'''
BUFFER_SIZE = 50000：这是用于打乱数据集的缓冲区大小。
在TensorFlow中，shuffle操作会将数据集中的元素随机排序，以便在训练模型时增加样本的多样性。缓冲区大小决定了在内存中保留多少数据用于打乱。
较大的缓冲区可以提供更好的随机性，但也会增加内存的使用。
'''
BUFFER_SIZE = 50000
'''
BATCH_SIZE = 64：这是每个批次中的样本数量。
在训练模型时，数据通常被分成小批量（batch），每个批次包含一定数量的样本。
这样可以更高效地利用内存，并且有助于模型的稳定训练。
'''
BATCH_SIZE = 64
'''
TAKE_SIZE = 5000：这个变量没有在代码中直接使用，但它可能用于限制数据集中处理的样本数量，例如，用于调试或测试目的。
'''
TAKE_SIZE = 5000

'''
遍历了labeled_data_sets列表中剩余的所有标记数据集，并将每个数据集与all_labeled_data合并。这是通过concatenate方法完成的，它将两个数据集合并为一个单一的数据集。
相当于从 [ [], [], []] -> [] ，整合数据集
'''
all_labeled_data = labeled_data_sets[0] # 第一个数据集
for labeled_dataset in labeled_data_sets[1:]: # 从第二个数据集开始遍历
    all_labeled_data = all_labeled_data.concatenate(labeled_dataset)


'''
在合并所有数据集之后，你使用shuffle方法对整个数据集进行打乱。
BUFFER_SIZE参数指定了用于打乱操作的缓冲区大小。
reshuffle_each_iteration=False参数意味着数据集只会在第一次迭代时被打乱，之后的迭代将不会再次打乱。这通常用于确保在每个epoch（训练周期）开始时数据被随机化，但在同一个epoch内保持固定的顺序。
'''
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

In [29]:
for ex in all_labeled_data.take(5):
    print(ex)

# 根据 labeler 函数的定义，返回的第一个tensor是string，文章的具体一行的内容，第二个tensor是tf.int64，第几个被迭代的文件
# (<tf.Tensor: shape=(), dtype=string, numpy=b'His good housekeeper answered, "Hector, since you bid me tell you'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)

(<tf.Tensor: shape=(), dtype=string, numpy=b'Ah! it were well if in the fishy deep'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"Thou slave of woman, manhood's counterfeit,">, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Fell to the ground, some from the grasp, and some'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'And jealous both lest I should also win'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Their dread approach, for to his tower they bent;'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


2024-04-15 17:56:20.669874: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
